In [24]:
import os
os.environ["THEANO_FLAGS"] = 'cuda.root=/usr/local/cuda,floatX=float32,device=gpu1,force_device=False,lib.cnmem=.75'

import theano
print(theano.config.device)

import mhcflurry, seaborn, numpy, pandas, pickle, sklearn, collections, scipy, time
import mhcflurry.dataset
import fancyimpute, locale

import sklearn.metrics
import sklearn.cross_validation

def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')
    
%matplotlib inline


gpu1


In [2]:
max_ic50 = 50000
min_peptides_to_consider_allele = 10
data_dir = "../data/"

In [3]:
all_train_data = mhcflurry.dataset.Dataset.from_csv(data_dir + "bdata.2009.mhci.public.1.txt")

In [4]:
imputed_train_data = all_train_data.impute_missing_values(
    fancyimpute.MICE(n_imputations=250, n_burn_in=50),
    min_observations_per_peptide=2,
    min_observations_per_allele=2
)


Dropping 12235 peptides with <2 observations
Dropping 9 alleles with <2 observations: ['ELA-A1', 'HLA-B2701', 'HLA-B3508', 'HLA-B44', 'HLA-E0101', 'Mamu-B04', 'Patr-A0602', 'Patr-B0901', 'Patr-B1701']
[MICE] Completing matrix with shape (19304, 97)
[MICE] Starting imputation round 1/300, elapsed time 0.024
[MICE] Starting imputation round 2/300, elapsed time 1.564
[MICE] Starting imputation round 3/300, elapsed time 2.855
[MICE] Starting imputation round 4/300, elapsed time 4.131
[MICE] Starting imputation round 5/300, elapsed time 5.399
[MICE] Starting imputation round 6/300, elapsed time 6.674
[MICE] Starting imputation round 7/300, elapsed time 7.941
[MICE] Starting imputation round 8/300, elapsed time 9.213
[MICE] Starting imputation round 9/300, elapsed time 10.482
[MICE] Starting imputation round 10/300, elapsed time 11.755
[MICE] Starting imputation round 11/300, elapsed time 13.023
[MICE] Starting imputation round 12/300, elapsed time 14.301
[MICE] Starting imputation round 13/

In [5]:
imputed_train_data.to_dataframe()

allele          peptide       affinity  \
allele    peptide                                                      
Mamu-A07  RELYLNSSNV        Mamu-A07       RELYLNSSNV    1615.211000   
          ICKAAMGLR         Mamu-A07        ICKAAMGLR    1684.829504   
          SAVTDRETDV        Mamu-A07       SAVTDRETDV    1691.783274   
          NSHQRSDSS         Mamu-A07        NSHQRSDSS    1694.957645   
          STATLCLGHH        Mamu-A07       STATLCLGHH    1682.656229   
          AIYVFCISLK        Mamu-A07       AIYVFCISLK    1659.680822   
          KPVDTSNSF         Mamu-A07        KPVDTSNSF    1632.351678   
          SIYIAVANCV        Mamu-A07       SIYIAVANCV    1631.715646   
          PWLTEKEAM         Mamu-A07        PWLTEKEAM    1698.113597   
          PSLPSPSR          Mamu-A07         PSLPSPSR    1678.871225   
          EEILGTVSW         Mamu-A07        EEILGTVSW    1559.525566   
          VRGGMVAPL         Mamu-A07        VRGGMVAPL    1704.502331   
          LTQRRTTPR         Mamu-A07        LTQRRTTPR    1617.113308   
          MSIMPVLAY         Mamu-A07        MSIMPVLAY    1584.375149   
          IVILFIMFM         Mamu-A07        IVILFIMFM    1658.351731   
          GLYCYFTHL         Mamu-A07        GLYCYFTHL    1581.397127   
          TILDDNLYK         Mamu-A07        TILDDNLYK    1667.000649   
          LYLYALIYFL        Mamu-A07       LYLYALIYFL    1609.864718   
          RGINDRNFW         Mamu-A07        RGINDRNFW    1694.400571   
          QLLALADRI         Mamu-A07        QLLALADRI    1696.888851   
          KTRMEDYYL         Mamu-A07        KTRMEDYYL    1697.055873   
          TVKPGNFNK         Mamu-A07        TVKPGNFNK    1596.731368   
          VIVPDIKLDA        Mamu-A07       VIVPDIKLDA    1694.371904   
          TISSESLVY         Mamu-A07        TISSESLVY    1699.968730   
          RVYAELAAL         Mamu-A07        RVYAELAAL    1593.762818   
          SQAELTSNCT        Mamu-A07       SQAELTSNCT    1664.570572   
          VTPLLKIL          Mamu-A07         VTPLLKIL    1658.983060   
          CRGEFLYCKM        Mamu-A07       CRGEFLYCKM    1643.095810   
          VIGLIVILFI        Mamu-A07       VIGLIVILFI    1630.718529   
          YTAVVKLVY         Mamu-A07        YTAVVKLVY    1568.170612   
...                              ...              ...            ...   
HLA-A0219 PVSDLYTSMR       HLA-A0219       PVSDLYTSMR     281.847354   
          YTDEVYDYL        HLA-A0219        YTDEVYDYL      71.905121   
          AITDNGPMPY       HLA-A0219       AITDNGPMPY    5381.516144   
          DTGNYILCY        HLA-A0219        DTGNYILCY    4483.233637   
          GSFCTQLNR        HLA-A0219        GSFCTQLNR     848.997986   
          SLVSSLWSMI       HLA-A0219       SLVSSLWSMI    4287.694356   
          SPRPEMQEF        HLA-A0219        SPRPEMQEF   61634.876926   
          FTNNEFTLS        HLA-A0219        FTNNEFTLS    5519.317320   
          HHIPNGVVW        HLA-A0219        HHIPNGVVW   21415.925021   
          YCNYTRFWYI       HLA-A0219       YCNYTRFWYI    1222.106966   
          PAEMLANID        HLA-A0219        PAEMLANID   21797.092415   
          NQFGSVPAL        HLA-A0219        NQFGSVPAL    6359.421750   
          YLIPSVTSL        HLA-A0219        YLIPSVTSL   20000.000000   
          TDSPETHHY        HLA-A0219        TDSPETHHY   42660.441140   
          GFLNEDHWF        HLA-A0219        GFLNEDHWF      59.541082   
          AATKRYPGVM       HLA-A0219       AATKRYPGVM   40131.279975   
          KLQDLTLRC        HLA-A0219        KLQDLTLRC     847.943608   
          SDLDMLTQS        HLA-A0219        SDLDMLTQS   15517.173699   
          EFCVDHPFIY       HLA-A0219       EFCVDHPFIY   65254.538684   
          APRELLQYI        HLA-A0219        APRELLQYI    9110.772894   
          RRQWVLAFR        HLA-A0219        RRQWVLAFR   21702.989962   
          ATRSHRPLI        HLA-A0219        ATRSHRPLI     935.816763   
          TATPAWDAL        HLA-A0219        TATPAWDAL   20000.000000  

In [6]:
validation_df = pandas.read_csv("../data/combined_test_BLIND_dataset_from_kim2013.csv")
validation_df

,allele,peptide,length,meas,netmhc,netmhcpan,smmpmbec_cpp
0,H-2-DB,AAACNVATA,9,657.657837,154.881662,711.213514,438.530698
1,H-2-DB,AAFEFVYV,8,30831.879502,6456.542290,785.235635,10351.421667
2,H-2-DB,AAFVNDYSL,9,77.446180,17.458222,7.516229,28.054336
3,H-2-DB,AAIANQAAV,9,1.999862,9.638290,9.749896,25.703958
4,H-2-DB,AAIANQAVV,9,1.517050,8.550667,8.336812,28.773984
5,H-2-DB,AAIENYVRF,9,37.844258,252.348077,114.815362,187.068214
6,H-2-DB,AAINFITTM,9,3.155005,199.986187,389.045145,200.909281
7,H-2-DB,AAIPAPPPI,9,3243.396173,1059.253725,493.173804,295.120923
8,H-2-DB,AAKLNRPPL,9,654.636174,66.374307,77.268059,38.459178
9,H-2-DB,AALDMVDAL,9,229.614865,547.015963,597.035287,225.423921


In [7]:
validation_allele_counts = validation_df.allele.value_counts()
train_allele_counts = all_train_data._df.allele.value_counts()
print(validation_allele_counts)
print(train_allele_counts)

HLA-A0201    2126
HLA-A2601    1333
HLA-B0801     940
HLA-B5101     854
HLA-B5701     815
HLA-B0702     813
HLA-A0301     811
HLA-A3101     724
HLA-A1101     723
HLA-A0101     696
HLA-A0206     682
HLA-A6802     669
HLA-A3001     660
HLA-A0203     651
HLA-B3901     641
HLA-B1501     633
HLA-B1517     582
HLA-A2402     573
H-2-DB        564
H-2-KB        558
HLA-B3501     542
HLA-A6801     527
HLA-B0802     509
HLA-B1801     503
HLA-B5301     485
HLA-A3301     473
HLA-A6901     470
HLA-B1509     466
HLA-A3201     449
HLA-B5801     445
HLA-B2703     441
HLA-A3002     420
HLA-A2501     416
HLA-A2602     413
HLA-B4402     411
HLA-B4001     407
HLA-A2301     391
Mamu-A02      388
HLA-A8001     379
HLA-B4601     378
HLA-B4403     378
HLA-B3801     351
HLA-B2705     314
HLA-A2603     312
Mamu-A01      274
HLA-B0803     234
H-2-KD        229
HLA-B1503     165
HLA-A0202     126
HLA-A2902     118
HLA-B5401      79
HLA-B4002      74
HLA-B4501      65
Name: allele, dtype: int64
HLA-A0201     9565


In [35]:
alleles = sorted(train_allele_counts.index[
    (train_allele_counts >= min_peptides_to_consider_allele)
    & (train_allele_counts.index.isin(validation_allele_counts.index))
], key=lambda allele: -1 * train_allele_counts[allele])
alleles
not_dropped = []
for allele in alleles:
    sub = all_train_data.get_allele(allele)._df
    if (sub.affinity < 500).sum() < 5 or (sub.affinity > 500).sum() < 5:
        print("Dropping allele %s" % allele)
    else:
        not_dropped.append(allele)
alleles = not_dropped
len(alleles)

Dropping allele HLA-B2703
Dropping allele HLA-B3801


51

In [36]:
dropout_probabilities = [0.5]
embedding_output_dims_and_layer_sizes_list = [(32, [64])] # , (8, [4])]
activations = ["tanh"]

models_params_list = []

for model_num in range(10):
    for fraction_negative in [.2]:
        for impute in [True]:
            for dropout_probability in dropout_probabilities:
                for (embedding_output_dim, layer_sizes) in embedding_output_dims_and_layer_sizes_list:
                    for activation in activations:
                        models_params_list.append(dict(
                            fraction_negative=fraction_negative,
                            impute=impute,
                            dropout_probability=dropout_probability,  
                            embedding_output_dim=embedding_output_dim,
                            layer_sizes=layer_sizes,
                            activation=activation))

print("%d models" % len(models_params_list))
models_params_explored = set.union(*[set(x) for x in models_params_list])
models_params_explored


10 models


{'activation',
 'dropout_probability',
 'embedding_output_dim',
 'fraction_negative',
 'impute',
 'layer_sizes'}

In [37]:
def make_scores(ic50_y, ic50_y_pred, sample_weight=None, threshold_nm=500):     
    y_pred = mhcflurry.regression_target.ic50_to_regression_target(ic50_y_pred, max_ic50)
    try:
        auc = sklearn.metrics.roc_auc_score(ic50_y <= threshold_nm, y_pred, sample_weight=sample_weight)
    except ValueError:
        auc = numpy.nan
    try:
        f1 = sklearn.metrics.f1_score(ic50_y <= threshold_nm, ic50_y_pred <= threshold_nm, sample_weight=sample_weight)
    except ValueError:
        f1 = numpy.nan
    try:
        tau = scipy.stats.kendalltau(ic50_y_pred, ic50_y)[0]
    except ValueError:
        tau = numpy.nan
    
    return dict(
        auc=auc,
        f1=f1,
        tau=tau,
    )    


In [38]:
models_and_scores = {}
validation_df_with_mhcflurry = validation_df.copy()


In [ ]:
# train and test models, adding columns to validation_df_with_mhcflurry
pandas.DataFrame(models_params_list).to_csv("../data/validation_models.csv", index=False)

def make_and_fit_model(allele, original_params):
    params = dict(original_params)
    impute = params["impute"]
    del params["impute"]
    
    fraction_negative = params["fraction_negative"]
    del params["fraction_negative"]
    
    model = mhcflurry.Class1BindingPredictor.from_hyperparameters(max_ic50=max_ic50, **params)
    print("Fitting model for allele %s (%d + %d): %s" % (
            allele,
            len(all_train_data.groupby_allele_dictionary()[allele]),
            len(imputed_train_data.groupby_allele_dictionary()[allele]),
            str(original_params)))
    t = -time.time()
    model.fit_dataset(
        all_train_data.get_allele(allele),
        pretraining_dataset=imputed_train_data.get_allele(allele) if impute else None,
        verbose=False,
        batch_size=128,
        n_training_epochs=250,
        n_random_negative_samples=int(fraction_negative * len(all_train_data.get_allele(allele))))
    t += time.time()
    print("Trained in %d sec" % t)
    return model

for (i, allele) in enumerate(alleles):
    if allele not in validation_df_with_mhcflurry.allele.unique():
        print("Skipping allele %s: not in test set" % allele)
        continue
    if allele in models_and_scores:
        print("Skipping allele %s: already done" % allele)
        continue
    values_for_allele = []
    for (j, params) in enumerate(models_params_list):
        print("Allele %d model %d" % (i, j))
        model = make_and_fit_model(allele, params)
        predictions = model.predict(
            list(validation_df_with_mhcflurry.ix[validation_df_with_mhcflurry.allele == allele].peptide))
        print("test set size: %d" % len(predictions))
        validation_df_with_mhcflurry.loc[(validation_df_with_mhcflurry.allele == allele),
                                         ("mhcflurry %d" % j)] = predictions
        scores = make_scores(validation_df_with_mhcflurry.ix[validation_df.allele == allele].meas,
                            predictions)
        print(scores)
        values_for_allele.append((params, scores))
        
    models_and_scores[allele] = values_for_allele
    
    # Write out all data after each allele.
    validation_df_with_mhcflurry_results = validation_df_with_mhcflurry.ix[validation_df_with_mhcflurry.allele.isin(models_and_scores)]
    validation_df_with_mhcflurry_results.to_csv("../data/validation_predictions_full.csv", index=False)
    
    scores_df = collections.defaultdict(list)
    predictors = validation_df_with_mhcflurry_results.columns[4:]

    for (allele, grouped) in validation_df_with_mhcflurry_results.groupby("allele"):
        scores_df["allele"].append(allele)
        scores_df["test_size"].append(len(grouped.meas))
        for predictor in predictors:
            scores = make_scores(grouped.meas, grouped[predictor])
            for (key, value) in scores.items():
                scores_df["%s_%s" % (predictor, key)].append(value)

    scores_df = pandas.DataFrame(scores_df)
    scores_df["train_size"] = [
        len(all_train_data.groupby_allele_dictionary()[a])
        for a in scores_df.allele
    ]

    scores_df.index = scores_df.allele
    scores_df.to_csv("../data/validation_scores.csv", index=False)
        


Allele 0 model 0
Fitting model for allele HLA-A0201 (9565 + 19304): {'impute': True, 'dropout_probability': 0.5, 'embedding_output_dim': 32, 'layer_sizes': [64], 'fraction_negative': 0.2, 'activation': 'tanh'}
Trained in 52 sec
test set size: 2126
{'tau': 0.62557849460708048, 'auc': 0.92894540014958871, 'f1': 0.87652173913043485}
Allele 0 model 1
Fitting model for allele HLA-A0201 (9565 + 19304): {'impute': True, 'dropout_probability': 0.5, 'embedding_output_dim': 32, 'layer_sizes': [64], 'fraction_negative': 0.2, 'activation': 'tanh'}
Trained in 52 sec
test set size: 2126
{'tau': 0.62636919613143582, 'auc': 0.9298901235887026, 'f1': 0.867288060363959}
Allele 0 model 2
Fitting model for allele HLA-A0201 (9565 + 19304): {'impute': True, 'dropout_probability': 0.5, 'embedding_output_dim': 32, 'layer_sizes': [64], 'fraction_negative': 0.2, 'activation': 'tanh'}
Trained in 52 sec
test set size: 2126
{'tau': 0.62753539061712893, 'auc': 0.92956245325355269, 'f1': 0.87570867858700396}
Allele 

In [ ]:
print_full(scores_df[["train_size", "test_size"]].sort("train_size", inplace=False))

In [ ]:
2